In [ ]:
import os
import networkx as nx
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# --- 1. 環境変数の読み込み ---
# これが .env ファイルを探して、中身を読み込む魔法の呪文です
loaded = dotenv.load_dotenv()

# 読み込み確認（キーそのものは表示せず、読み込めたかだけチェック）
if not loaded:
    print("⚠️ .envファイルが見つかりません。作成しましたか？")
else:
    print("✅ .envファイルを読み込みました。")

# --- 2. モデルの準備 ---
# api_key 引数を指定しなくても、自動的に環境変数の OPENAI_API_KEY を使ってくれます
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# --- 3. プロンプト定義 ---
extract_prompt = ChatPromptTemplate.from_template("""
あなたは凄腕のデータサイエンティストです。
以下のテキストから、主要な「エンティティ(Entity)」と、
それらの間の「関係性(Relation)」を抽出してください。

出力は以下のJSON形式のみでお願いします（Markdown記法は不要）：
[
  {{"source": "エンティティA", "target": "エンティティB", "relation": "関係性の説明"}}
]

テキスト:
{text}
""")

# --- 4. テストデータ ---
text_chunk = """
プロジェクト「Titan」は2024年に開始されたルミナイ社の次世代AI基盤開発プロジェクトです。
リーダーの宮脇は、このプロジェクトにおいてコスト削減を最優先事項としています。
一方、マーケティング部の田中はTitanのリリース遅延に懸念を示しており、
宮脇と田中の間では定例会議で激しい議論が行われました。
"""

# --- 5. 実行処理 ---
print("🤖 GPT-4oに抽出を依頼中...")

# JSONパーサーを通して構造化データとして受け取る
chain = extract_prompt | llm | JsonOutputParser()

try:
    triplets = chain.invoke({"text": text_chunk})
    print(f"🔍 抽出結果 ({len(triplets)}件):")
    for t in triplets:
        print(f" - {t['source']} --[{t['relation']}]--> {t['target']}")

    # --- 6. グラフ構築 ---
    G = nx.Graph()
    for item in triplets:
        G.add_edge(item['source'], item['target'], relation=item['relation'])

    print("-" * 30)
    print(f"🕸️  グラフ構築完了！")
    print(f"   ノード数: {G.number_of_nodes()}")
    print(f"   エッジ数: {G.number_of_edges()}")
    print("   (NetworkXオブジェクト 'G' に格納されました)")

except Exception as e:
    print(f"エラーが発生しました: {e}")


In [ ]:
# NetworkX標準の機能でコミュニティ検出（Greedy Modularity法）
# ※論文ではLeidenを使いますが、簡易版としてこれで十分雰囲気はつかめます。
communities = list(nx.community.greedy_modularity_communities(G))

print(f"🧩 {len(communities)}個のコミュニティを検出しました！")

# 各コミュニティをLLMに「要約」させる
summarize_prompt = ChatPromptTemplate.from_template("""
以下の情報は、あるコミュニティ（関連性の高いグループ）の要素リストです。
このグループが全体として「何を表しているか」「どんな状況か」を要約してください。

要素と関係性:
{context}
""")

summarize_chain = summarize_prompt | llm
community_summaries = []

for i, community in enumerate(communities):
    # コミュニティ内のエッジ（関係性）を集める
    subgraph = G.subgraph(community)
    context_text = [f"{u} -> {v}: {d['relation']}" for u, v, d in subgraph.edges(data=True)]
    
    # 要約実行
    summary = summarize_chain.invoke({"context": "\n".join(context_text)})
    community_summaries.append(summary.content)
    print(f"📝 コミュニティ{i+1}要約: {summary.content}")


In [ ]:
query = "社内のプロジェクトにおける、部門間の対立構造について教えて"

# 要約だけをコンテキストとして渡す
answer_prompt = ChatPromptTemplate.from_template("""
以下の「コミュニティ要約」を参考にして、ユーザーの質問に答えてください。
個別の事実だけでなく、全体的な傾向や構造に触れること。

コミュニティ要約:
{summaries}

質問: {question}
""")

rag_chain = answer_prompt | llm
response = rag_chain.invoke({
    "summaries": "\n".join(community_summaries),
    "question": query
})

print("\n--- 🤖 GraphRAGによる回答 ---")
print(response.content)
